# MORTGAGE BACKED SECURITIES

IMPORTING LIBRARIES

In [13]:
import pandas as pd
import scipy
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OrdinalEncoder
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.ensemble import ExtraTreesClassifier
from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score, confusion_matrix, roc_auc_score, roc_curve
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestClassifier
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor

ImportError: DLL load failed while importing _path: The specified module could not be found.

LOADING DATA

In [ ]:
df=pd.read_csv(r'C:\Users\wania_96\Desktop\LoanExport\LoanExport.csv')

In [ ]:
df.head()

EDA

In [ ]:
df.describe()

In [ ]:
df.info()

In [ ]:
print(df.isnull().sum().sort_values(ascending=False),"\n\n",df.isnull().sum()/df.shape[0] *100)

In [ ]:
df['SellerName'].mode().values[0]

In [ ]:
df['SellerName'] = df['SellerName'].fillna(df['SellerName'].dropna().mode().values[0])

In [ ]:
df['SellerName'].isnull().sum()

In [ ]:
df["MSA"].unique()

In [ ]:
df[df["MSA"]!='X    ']["MSA"].astype(str).astype(int)

In [ ]:
df[df["MSA"]!='X    ']["MSA"].astype(str).astype(int).mean()

In [ ]:
mean_msa=round(df[df["MSA"]!='X    ']["MSA"].astype(str).astype(int).mean(),0)

In [ ]:
mean_msa

In [ ]:
df["MSA"]= df["MSA"].replace('X    ',mean_msa)

In [ ]:
df["MSA"]=pd.to_numeric(df["MSA"])

In [ ]:
df['Channel'].value_counts()

In [ ]:
df =  df[df["FirstTimeHomebuyer"]!= 'X']

In [ ]:
df['PPM'].value_counts()

In [ ]:
df =df[df['PPM']!='X']

In [ ]:
df['PPM'].value_counts()

In [ ]:
df.PropertyType.value_counts()

In [ ]:
df =df[df['PropertyType']!='X']

In [ ]:
df['NumBorrowers'] = pd.to_numeric(df['NumBorrowers'], errors='coerce').fillna(0).astype(int)

In [ ]:
df = df[df['NumBorrowers']!= 0]

In [ ]:
df.NumBorrowers.value_counts()

In [ ]:
 df.select_dtypes('object')

LABELLING

In [ ]:
df["aFirstTimeHomebuyer"] = df['FirstTimeHomebuyer'].map({"N":0,"Y":1})
df.head()

In [ ]:
df['LTV_Range'] = ""
for index, value in df['LTV'].iteritems():
    if value >= 0 and value < 25:
        df.at[index, 'LTV_Range'] = 'Low'
    elif value >= 25 and value < 50:
        df.at[index, 'LTV_Range'] = 'Medium'
    elif value >= 50 and value < 75:
        df.at[index, 'LTV_Range'] = 'High'
    elif value >= 75 and value <= 100:
        df.at[index, 'LTV_Range'] = ' Very High'
    else:
        df.at[index, 'LTV_Range'] = 'Invalid value'

In [ ]:
df['LTV'].value_counts().index

In [ ]:
df['DTI_Range'] = ""

for index, value in df['DTI'].iteritems():
    if value >= 0 and value < 25:
        df.at[index, 'DTI_Range'] = 'Low'
    elif value >= 25 and value < 50:
        df.at[index, 'DTI_Range'] = 'Medium'
    elif value >= 50 and value < 75:
        df.at[index, 'DTI_Range'] = 'High'
    elif value >= 75 and value < 100:
        df.at[index, 'DTI_Range'] = ' Very High'
    else:
        df.at[index, 'DTI_Range'] = 'Invalid value'

In [ ]:
df['CreditRange'] = ""

for index, value in df['CreditScore'].iteritems():
    if value >= 0 and value < 650:
        df.at[index, 'CreditRange'] = 'Poor'
    elif value >= 650 and value < 700:
        df.at[index, 'CreditRange'] = 'Fair'
    elif value >= 700 and value < 750:
        df.at[index, 'CreditRange'] = 'Good'
    elif value >= 750 and value <= 900:
        df.at[index, 'CreditRange'] = 'Excellent'
    else:
        df.at[index, 'CreditRange'] = 'Invalid value'

In [ ]:
df['MIP'].value_counts()

In [ ]:
df['MIP_Range'] = ""

for index, value in df['MIP'].iteritems():
    if value >= 0 and value < 25:
        df.at[index, 'MIP_Range'] = 'Low'
    elif value >= 25 and value < 50:
        df.at[index, 'MIP_Range'] = 'Medium'
    elif value >= 50 and value < 75:
        df.at[index, 'MIP_Range'] = 'High'
    else:
        df.at[index, 'MIP_Range'] = 'Invalid value'

In [ ]:
df['RepPayRange'] = ""
for index, value in df['MonthsInRepayment'].iteritems():
    if value >= 0 and value < 48:
        df.at[index, 'RepPayRange'] = '0-4yrs'
    elif value >= 48 and value < 96:
        df.at[index, 'RepPayRange'] = '4-8yrs'
    elif value >= 96 and value < 144:
        df.at[index, 'RepPayRange'] = '8-12yrs'
    elif value >= 144 and value < 192:
        df.at[index, 'RepPayRange'] = '12-16yrs'
    elif value >= 192 and value < 240:
        df.at[index, 'RepPayRange'] = '16-20yrs'
    else:
        df.at[index, 'RepPayRange'] = 'Invalid value'

In [ ]:
df["RepPayRange"].value_counts()

In [ ]:
df['FirstPaymentDate'] = pd.to_datetime(df['FirstPaymentDate'],format='%Y%m')
df['MaturityDate'] = pd.to_datetime(df['MaturityDate'], format='%Y%m')
df['Month_Difference'] = (df['MaturityDate'].dt.to_period('M') - df['FirstPaymentDate'].dt.to_period('M')).apply(lambda x: x.n) 

In [ ]:
def calculate_emi(OrigUPB, OrigInterestRate, OrigLoanTerm):
    r = (OrigInterestRate / 12) / 100  # Convert annual interest rate to monthly rate
    n = OrigLoanTerm  # Total number of monthly installments
    
    emi = (OrigUPB * r * (1 + r) ** n) / ((1 + r) ** n - 1)
    return round(emi, 2)  # Round the EMI to 2 decimal places
# Calculate EMI and create a new column for EMI
df['monthly_payment'] = calculate_emi(df['OrigUPB'], df['OrigInterestRate'], df['OrigLoanTerm'])

In [ ]:
df["total_payment"] = df['monthly_payment']*df["OrigLoanTerm"]
df['total_payment'] = df['total_payment'].round(2)
df["interest_amount"] = df["total_payment"]-df["OrigUPB"]
df['interest_amount'] = df['interest_amount'].round(2)

In [ ]:
df["monthly_income"] = (df["monthly_payment"]//df["DTI"])* 100
df['monthly_income'] = df['monthly_income'].round(2)

In [ ]:
df["monthly_rate"] = df["OrigInterestRate"]/1200
df['monthly_rate'] = df['monthly_rate'].round(4)

In [ ]:
def principle(monthly_rate,OrigUPB,monthly_payment,MonthsInRepayment):
    for i in range(MonthsInRepayment):
        interest = monthly_rate*OrigUPB
        p=monthly_payment-interest
    return round (p,2)
    

df["monthly_priciple amount"] = np.vectorize(principle)(df["monthly_rate"],df["OrigUPB"],df["monthly_payment"],df["MonthsInRepayment"])

In [ ]:
def principle(monthly_rate,OrigUPB,monthly_payment,MonthsInRepayment):
    remaining_principal = OrigUPB
    for i in range(MonthsInRepayment):
        interest = monthly_rate*OrigUPB
        p=monthly_payment-interest
        remaining_principal =  remaining_principal- p
    return round (remaining_principal,2)
    

df["principal_amount_remaining"] = np.vectorize(principle)(df["monthly_rate"],df["OrigUPB"],df["monthly_payment"],df["MonthsInRepayment"])

In [ ]:
def principle(monthly_rate,OrigUPB,monthly_payment,MonthsInRepayment):
    remaining_principal = OrigUPB
    for i in range(MonthsInRepayment):
        interest = monthly_rate*OrigUPB
        p=monthly_payment-interest
        remaining_principal =  remaining_principal- p
        priciple_amount_paid = OrigUPB - remaining_principal
    return round (priciple_amount_paid,2)
    

df["priciple_amount_paid"] = np.vectorize(principle)(df["monthly_rate"],df["OrigUPB"],df["monthly_payment"],df["MonthsInRepayment"])

In [ ]:
df.head()

In [ ]:
def prepay(dti,income):
    if(dti<40):
        p=income/2
    else:
        p=income*3/4
    return p
df["prepayment"]=np.vectorize(prepay)(df["DTI"],df["monthly_income"]*24)
df["prepayment"]=df["prepayment"]-(df["monthly_payment"]*24)

VISUALIZING

In [ ]:
plt.figure(figsize=(6,4))
sns.countplot(x='aFirstTimeHomebuyer' , data=df)
plt.xlabel('First Time Homebuyer')
plt.show()

In [ ]:
plt.figure(figsize=(6,4))
sns.countplot(x='Occupancy' , data=df)
plt.xlabel('Occupancy')
plt.show()

In [ ]:
plt.figure(figsize=(6,4))
sns.countplot(x='Channel' , data=df)
plt.xlabel('Channel', fontsize=14)
plt.show()

In [ ]:
plt.figure(figsize=(6,4))
sns.countplot(x='PPM' , data=df)
plt.xlabel('PPM')
plt.show()

In [ ]:
plt.figure(figsize=(25,5))
sns.countplot(x='ServicerName', data=df)
plt.xlabel('ServicerName')
plt.xticks(rotation='vertical')
plt.show()

In [ ]:
plt.figure(figsize=(6,3))
sns.countplot(x='PropertyType' , data=df)
plt.xlabel('PropertyType', fontsize=14)
plt.show()

In [ ]:
plt.figure(figsize=(6,3))
sns.countplot(x='NumBorrowers' , data=df)
plt.xlabel('NumBorrowers', fontsize=14)
plt.show()

In [ ]:
plt.figure(figsize=(16,3))
sns.countplot(x='ServicerName', data=df)
plt.xlabel('ServicerName')
plt.xticks(rotation='vertical')
plt.show()

In [ ]:
plt.figure(figsize=(10,5))
sns.barplot(x='aFirstTimeHomebuyer' ,y='EverDelinquent', data=df)
plt.xlabel('FirstTimeHomebuyer', fontsize=14)
plt.ylabel('EverDelinquent',fontsize=14)
plt.legend()
plt.show()

ENCODING

In [ ]:
encoder=LabelEncoder()

In [ ]:
df['PPM']= encoder.fit_transform(df['PPM'])
df['LoanPurpose']= encoder.fit_transform(df['LoanPurpose'])
df['PropertyState']= encoder.fit_transform(df['PropertyState'])
df['ServicerName']= encoder.fit_transform(df['ServicerName'])
df['PropertyType']= encoder.fit_transform(df['PropertyType'])
df['Channel']= encoder.fit_transform(df['Channel'])
df['SellerName']= encoder.fit_transform(df['SellerName'])
df['LTV_Range']= encoder.fit_transform(df['LTV_Range'])
df['DTI_Range']= encoder.fit_transform(df['DTI_Range'])
df['CreditRange']= encoder.fit_transform(df['CreditRange'])
df['MIP_Range']= encoder.fit_transform(df['MIP_Range'])
df['RepPayRange']= encoder.fit_transform(df['RepPayRange'])
df['Month_Difference']= encoder.fit_transform(df['Month_Difference'])

In [ ]:
df_city = pd.get_dummies(df['Occupancy'],drop_first = True)

In [ ]:
df=pd.concat([df,df_city],axis=1)

In [ ]:
df.drop('Occupancy',axis=1,inplace=True)

In [ ]:
df.head()

In [ ]:
df.drop(['PostalCode','LoanSeqNum',"ProductType"],axis=1, inplace=True)

In [ ]:
df.drop(['FirstTimeHomebuyer',"DTI","LTV",'MIP','MonthsInRepayment',"CreditScore","FirstPaymentDate","MaturityDate"],axis=1, inplace=True)

FEATURE ENGINEERING

In [ ]:
x=df.drop("prepayment",axis=1)
y=df["prepayment"]
vif = pd.DataFrame()
vif['Features'] = x.columns
vif['VIF'] = [variance_inflation_factor(x.values, i) for i in range(x.shape[1])]
print(vif)

In [ ]:
X1=x.drop("OrigLoanTerm",axis=1)

In [ ]:
vif = pd.DataFrame()
vif['Features'] = X1.columns
vif['VIF'] = [variance_inflation_factor(X1.values, i) for i in range(X1.shape[1])]

print(vif)

In [ ]:
X2=X1.drop("Month_Difference",axis=1)

In [ ]:
vif = pd.DataFrame()
vif['Features'] = X2.columns
vif['VIF'] = [variance_inflation_factor(X2.values, i) for i in range(X2.shape[1])]
print(vif)

In [ ]:
X3=X2.drop("OCLTV",axis=1)

In [ ]:
vif = pd.DataFrame()
vif['Features'] = X3.columns
vif['VIF'] = [variance_inflation_factor(X3.values, i) for i in range(X3.shape[1])]
print(vif)

In [ ]:
X4=X3.drop("OrigInterestRate",axis=1)

In [ ]:
vif = pd.DataFrame()
vif['Features'] = X4.columns
vif['VIF'] = [variance_inflation_factor(X4.values, i) for i in range(X4.shape[1])]
print(vif)

In [ ]:
X5=X4.drop("O",axis=1)

In [ ]:
vif = pd.DataFrame()
vif['Features'] = X5.columns
vif['VIF'] = [variance_inflation_factor(X5.values, i) for i in range(X5.shape[1])]
print(vif)

In [ ]:
X6=X5.drop("Units",axis=1)

In [ ]:
vif = pd.DataFrame()
vif['Features'] = X6.columns
vif['VIF'] = [variance_inflation_factor(X6.values, i) for i in range(X6.shape[1])]
print(vif)

In [ ]:
X7=X6.drop("Channel",axis=1)

In [ ]:
vif = pd.DataFrame()
vif['Features'] = X7.columns
vif['VIF'] = [variance_inflation_factor(X7.values, i) for i in range(X7.shape[1])]
print(vif)

In [ ]:
X8=X7.drop("PropertyType",axis=1)

In [ ]:
vif = pd.DataFrame()
vif['Features'] = X8.columns
vif['VIF'] = [variance_inflation_factor(X8.values, i) for i in range(X8.shape[1])]
print(vif)

In [ ]:
X9=X8.drop("NumBorrowers",axis=1)

In [ ]:
vif = pd.DataFrame()
vif['Features'] = X9.columns
vif['VIF'] = [variance_inflation_factor(X9.values, i) for i in range(X9.shape[1])]
print(vif)

In [ ]:
X10=X9.drop("MIP_Range",axis=1)

In [ ]:
vif = pd.DataFrame()
vif['Features'] = X10.columns
vif['VIF'] = [variance_inflation_factor(X10.values, i) for i in range(X10.shape[1])]
print(vif)

In [ ]:
scaler = StandardScaler()
scaler_fit = scaler.fit_transform(X10)
X=pd.DataFrame(scaler_fit, columns = X10.columns)

In [ ]:
X

In [ ]:
pca = PCA()
pca.fit_transform(X)

In [ ]:
principalComponents = pca.fit_transform(X)

In [ ]:
plt.figure()
plt.plot(np.cumsum(pca.explained_variance_ratio_))
plt.xlabel('Number of Components')
plt.ylabel('Variance (%)') #for each component
plt.title('Explained Variance')
plt.show();

In [ ]:
new_data = pd.DataFrame(principalComponents)

In [ ]:
pca = PCA(n_components=12)
final_data = pca.fit_transform(new_data)
principal_Df = pd.DataFrame(data = final_data
             , columns = ['PC1','PC2','PC3','PC4','PC5','PC6','PC7','PC8','PC9','PC10','PC11','PC12'])

In [ ]:
principal_Df

In [ ]:
X = principal_Df
Y = df["EverDelinquent"]

SPLITTING

In [ ]:
X_train,X_test,Y_train,Y_test = train_test_split(X,Y, test_size=0.3, random_state=42)

MODELING

In [ ]:
lr = LogisticRegression()
lr.fit(X_train,Y_train)
Y_pred = lr.predict(X_test)

In [ ]:
print(accuracy_score(Y_train,lr.predict(X_train)))

In [ ]:
print('accuracy score using logistic regression',accuracy_score(Y_test,Y_pred))


In [ ]:
print('classification report',classification_report(Y_test,Y_pred))


In [ ]:
rfc = RandomForestClassifier()
rfc.fit(X_train,Y_train)

In [ ]:
accuracy_score(Y_train,rfc.predict(X_train))

In [ ]:
print('accuracy score using random forest model',accuracy_score(Y_test, rfc.predict(X_test)))

In [ ]:
print('classification report', classification_report(Y_test, rfc.predict(X_test)))